<a href="https://colab.research.google.com/github/Buitrongquy/Age_Classification/blob/main/Age_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
#dataset = Dataset.list_files("/content/drive/MyDrive/FILE/Face_data/UTKFace/*.jpg.chip.jpg")

In [ ]:
train_dataset = tf.data.Dataset.list_files("/content/drive/MyDrive/FILE/Face_data/train_dataset/*.jpg")
val_dataset = tf.data.Dataset.list_files("/content/drive/MyDrive/FILE/Face_data/val_dataset/*.jpg")
test_dataset = tf.data.Dataset.list_files("/content/drive/MyDrive/FILE/Face_data/test_dataset/*.jpg")

In [ ]:
import numpy as np
def get_label(file_path):
  name = tf.strings.split(file_path,'/')[-1]
  name = tf.strings.split(name,'.')[0]
  age = int(tf.strings.split(name,'_')[2])
  group = (age - 1)//5
  label = list(np.zeros(24, dtype=np.float32))
  for i in range(24):
    if i == group:
      label[i] = 1.0
  return tf.convert_to_tensor(label)

In [ ]:
for path in train_dataset.take(10):
  print(get_label(path))


tf.Tensor([1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], shape=(24,), dtype=float32)
tf.Tensor([0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], shape=(24,), dtype=float32)
tf.Tensor([0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], shape=(24,), dtype=float32)
tf.Tensor([0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], shape=(24,), dtype=float32)
tf.Tensor([0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], shape=(24,), dtype=float32)
tf.Tensor([0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], shape=(24,), dtype=float32)
tf.Tensor([0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], shape=(24,), dtype=float32)
tf.Tensor([0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.], shape=(24,), dtype=float32)
tf.Tensor([1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], shape=(24,)

In [ ]:
def preprocessing(file_path):
  img = tf.io.read_file(file_path)
  img = tf.io.decode_jpeg(img, channels=1)
  img = tf.image.resize(img, [100,100])
  img = tf.image.convert_image_dtype(img, dtype=tf.float32)
  label = get_label(file_path)

  return img, label



In [ ]:
#dataset = Dataset.list_files("/content/drive/MyDrive/FILE/Face_data/UTKFace/*.jpg.chip.jpg")
train_dataset = train_dataset.map(preprocessing).shuffle(13000).batch(32).prefetch(2)
val_dataset = val_dataset.map(preprocessing).batch(32).shuffle(2000)
test_dataset = test_dataset.map(preprocessing).batch(32).shuffle(2000)

In [ ]:
input = tf.keras.Input(shape=(100,100,1))

layer_1 = tf.keras.layers.Conv2D(10, 1, padding='same', activation='relu')(input)

layer_2 = tf.keras.layers.Conv2D(5, 1, padding='same', activation='relu')(input)
layer_2 = tf.keras.layers.Conv2D(5, 3, padding='same', activation='relu')(layer_2)

layer_3 = tf.keras.layers.Conv2D(5, 1, padding='same', activation='relu')(input)
layer_3 = tf.keras.layers.Conv2D(5, 5, padding='same', activation='relu')(layer_3)

layer_4 = tf.keras.layers.MaxPooling2D(pool_size=3, strides=(1,1), padding='same')(input)
layer_4 = tf.keras.layers.Conv2D(10, 1, padding='same', activation='relu')(layer_4)

inception = tf.keras.layers.concatenate([layer_1, layer_2, layer_3, layer_4], axis = 3)


In [ ]:
flatten = tf.keras.layers.Flatten()(inception)
desen = tf.keras.layers.Dense(800, activation='relu')(flatten)
desen = tf.keras.layers.Dense(200, activation='relu')(desen)
output = tf.keras.layers.Dense(24, activation='softmax')(desen)

In [ ]:
model = tf.keras.Model(input,output)

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 100, 100, 1)]        0         []                            
                                                                                                  
 conv2d_1 (Conv2D)           (None, 100, 100, 5)          10        ['input_1[0][0]']             
                                                                                                  
 conv2d_3 (Conv2D)           (None, 100, 100, 5)          10        ['input_1[0][0]']             
                                                                                                  
 max_pooling2d (MaxPooling2  (None, 100, 100, 1)          0         ['input_1[0][0]']             
 D)                                                                                           

In [ ]:
tf.keras.utils.plot_model(model)

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4, weight_decay=1e-3)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['acc'])

In [ ]:
path = '/content/drive/MyDrive/FILE/Face_data/model.h5'
earlystopping = tf.keras.callbacks.EarlyStopping(monitor='val_acc', min_delta=1e-4, patience=8, verbose=1, mode='max',restore_best_weights=True)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_acc', factor=0.5, patience=5, verbose=1, mode='max', min_delta=1e-2, min_lr=1e-9)
checkpointer = tf.keras.callbacks.ModelCheckpoint(filepath=path, monitor='val_acc', verbose=1, save_best_only=True, mode='max' )

In [ ]:
callbacks = [earlystopping, reduce_lr, checkpointer]

In [ ]:
model.fit(train_dataset,
          steps_per_epoch=len(train_dataset),
          epochs=10,
          verbose=1,
          validation_data=val_dataset,
          validation_steps=len(val_dataset),
          callbacks=callbacks,)

Epoch 1/10
383/383 [==============================] - ETA: 0s - loss: 0.0466 - acc: 0.9947
Epoch 1: val_acc did not improve from 0.79702
383/383 [==============================] - 54s 105ms/step - loss: 0.0466 - acc: 0.9947 - val_loss: 2.2870 - val_acc: 0.7856 - lr: 5.0000e-05
Epoch 2/10
383/383 [==============================] - ETA: 0s - loss: 0.0532 - acc: 0.9926
Epoch 2: val_acc did not improve from 0.79702
383/383 [==============================] - 52s 102ms/step - loss: 0.0532 - acc: 0.9926 - val_loss: 2.3191 - val_acc: 0.7951 - lr: 5.0000e-05
Epoch 3/10
383/383 [==============================] - ETA: 0s - loss: 0.0403 - acc: 0.9948
Epoch 3: val_acc did not improve from 0.79702
383/383 [==============================] - 51s 97ms/step - loss: 0.0403 - acc: 0.9948 - val_loss: 2.2522 - val_acc: 0.7932 - lr: 5.0000e-05
Epoch 4/10
383/383 [==============================] - ETA: 0s - loss: 0.0482 - acc: 0.9939
Epoch 4: val_acc did not improve from 0.79702
383/383 [=====================

In [ ]:
model = tf.keras.saving.load_model('/content/drive/MyDrive/FILE/Face_data/model.h5')

In [ ]:
model.evaluate(test_dataset)

82/82 [==============================] - 483s 12ms/step - loss: 2.3949 - acc: 0.7721


[2.3948867321014404, 0.7721374034881592]